<a href="https://colab.research.google.com/github/Sravani-05/LLM-finetuning-assignment/blob/main/G_quantization_GGUF_GGML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##In this colab, I have used GGML and GGUF for quantization of LLM

###Quantize llm with ggml

In [ ]:
!nvidia-smi

Sat Nov  4 01:06:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:          6
    Model:               85
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            3
    BogoMIPS:            4000.40
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [ ]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

Choosing model

In [ ]:
model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [ ]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_id,
                                           model_type="llama",
                                           #lib='avx2', for cpu use
                                           gpu_layers=130, #110 for 7b, 130 for 13b
                                           **config
                                           )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

LLAMA-2-7B-Chat tests

Tokenizer

In [ ]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [ ]:
tokens = llm.tokenize(prompt)

In [ ]:
tokens

Pipeline

In [ ]:
# 'pipeline' execution
llm(prompt, stream=False)

"\n\nI'm trying to learn the periodic table and I need some help!\n\nHere is what I have so far:\n\nHydrogen (H)\nHelium (He)\nLithium (Li)\nBeryllium (Be)\nBoron (B)\nCarbon (C)\nNitrogen (N)\nOxygen (O)\nFluorine (F)\nNeon (Ne)\n\nI can't seem to remember the rest of the elements!  Can you help me?\n\nHere is what I am trying to memorize:\n\n1. Hydrogen (H)\n2. Helium (He)\n3. Lithium (Li)\n4. Beryllium (Be)\n5. Boron (B)\n6. Carbon (C)\n7. Nitrogen (N)\n8. Oxygen (O)\n9. Fluorine (F)\n10. Neon (Ne)\n\nI hope this helps! Let me know if you have any other questions.\n\nBest,\n[Your Name]\n\nI'd be happy to help"

In [ ]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nA descoberta do Brasil é um evento histórico que ocorreu em 1500, quando a expedição de Pedro Álvares Cabral alcançou as costas do atual Brasil. A expedição partiu da Portugal em 2 de abril de 1500 e chegou ao Brasil em 22 de maio do mesmo ano.\n\nA descoberta do Brasil é creditada a Pedro Álvares Cabral, que foi o líder da expedição. Ele era um navegador português que havia participado de outras viagens para o leste da África e para a Índia. A expedição de Cabral foi a primeira a chegar ao Novo Mundo desde a viagem de Cristóvão Colombo em 1492.\n\nA descoberta do Brasil teve um impacto significativo na história do mundo, pois marcou o início da colonização europeia das Américas e ajudou a estabelecer o domínio português sobre a região.'

Generate with stream execution

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from ctransformers import AutoModelForCausalLM
import time

# Load your model (this is a placeholder – you would need to configure your actual model)
model_id = model_id  # Replace with your actual model ID
llm = AutoModelForCausalLM.from_pretrained(model_id, model_type="llama", gpu_layers=130)

def generate_response(prompt):
    start = time.time()
    NUM_TOKENS = 0
    response = ""
    tokens = llm.tokenize(prompt)
    # Generate tokens (this is pseudocode; replace with your actual generation method)
    for token in llm.generate(tokens):
        response += llm.detokenize(token)
        NUM_TOKENS += 1
    time_generate = time.time() - start
    return response, NUM_TOKENS, time_generate

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs=["text", "number", "number"]
)

iface.launch(debug = True)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://98845c91ded969f0f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://98845c91ded969f0f4.gradio.live


In [ ]:
import gradio as gr
from ctransformers import AutoModelForCausalLM
import time

# Load your model (this is a placeholder – you would need to configure your actual model)
model_id = model_id  # Replace with your actual model ID
llm = AutoModelForCausalLM.from_pretrained(model_id, max_new_tokens= 1000, model_type="llama", gpu_layers=130)

def generate_response(prompt):
    start = time.time()
    NUM_TOKENS = 0
    response = ""
    tokens = llm.tokenize(prompt)
    # Generate tokens (this is pseudocode; replace with your actual generation method)
    for token in llm.generate(tokens):
        response += llm.detokenize(token)
        NUM_TOKENS += 1
    time_generate = time.time() - start
    return response, NUM_TOKENS, time_generate

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs=["text", "number", "number"]
)

iface.launch(debug = True)


iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs=["text", "number", "number"]
)

iface.launch(debug = True)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://eec07657f1b328d937.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://eec07657f1b328d937.gradio.live
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c73f85629b892f50e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://c73f85629b892f50e6.gradio.live


### Pipeline

In [ ]:
# 'pipeline' execution
llm(prompt, stream=False)

'\n\nI\'m trying to learn these in order to pass my chemistry class, and I find it hard to remember them all. Any help would be appreciated!\n\nThank you!"\n\nOf course, I\'d be happy to help! Here is a poem to help you remember the first 10 elements on the periodic table:\n\nHydrogen is number one, a lightest element of all,\nBoring and unreactive, it\'s hard to recall.\nHelium\'s next, with a voice so high and thin,\nIn balloons it\'s used, and makes them fly within.\nLithium follows, with a charge so strong,\nIt\'s in your phone and other devices all day long.\nBeryllium\'s next, a element so rare,\nIn gemstones it\'s found, and adds beauty there.\nBoron\'s the next, with a name so fun,\nIn insecticides it\'s used, to keep them done.\nCarbon\'s the king, of all elements we know,\nFrom diamonds to coal, it\'s found in many a show.\nNitrogen\'s'

In [ ]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nA descoberta do Brasil é um evento histórico que ocorreu em 1500, quando a expedição de Pedro Álvares Cabral alcançou as costas do atual Brasil. A expedição partiu da Portugal em 2 de abril de 1500 e chegou ao Brasil em 22 de maio do mesmo ano.\n\nA descoberta do Brasil é creditada a Pedro Álvares Cabral, que foi o líder da expedição. Ele era um navegador português que havia participado de outras viagens para o leste da África e para a Índia. A expedição de Cabral foi a primeira a chegar ao Novo Mundo desde a viagem de Cristóvão Colombo em 1492.\n\nO objetivo da expedição de Cabral era encontrar uma rota marítima para a Índia, mas quando chegou às costas do Brasil, ele descobriu um novo continente. A descoberta do Brasil teve um impacto significativo na história do mundo e marcou o início de uma nova era de exploração'

### Stream generation

In [ ]:
# LlAMA-2-13b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


I'm trying to learn these in order to pass my chemistry class and I would really appreciate your help!

Thank you!"

Here is a poem to help you remember the first 10 elements on the periodic table:

Hydrogen is number one, light as can be
Helium's next, with a balloon-like spree
Then comes Lithium, shining bright and bold
Beryllium's next, its strength untold
Boron's the next, with a tricky name
Carbon's the one that makes all life the same
Nitrogen's up, with an airy fame
Oxygen's next, for breathing we claim
Fluorine's last, with a twist of fame

I hope this helps you remember the first 10 elements on the periodic table!

----End Generation----
Num of generated tokens: 192
Time for complete generation: 26.46462845802307s
Tokens per secound: 7.2549667683618235
Time per token: 137.8366065522035ms


###Quantize large language model with GGUF

In [ ]:
!pip install ctransformers>=0.2.24

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

--2023-11-04 08:09:16--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1699344556&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTM0NDU1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkN

In [ ]:
from ctransformers import AutoModelForCausalLM, AutoConfig, Config
conf = AutoConfig(Config(temperature=0.7, repetition_penalty=1.1, batch_size=52,
                max_new_tokens=1024, context_length=2048))
llm = AutoModelForCausalLM.from_pretrained("/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                                           model_type="mistral", config = conf)

In [ ]:
prompt = "quel est le sens de la rose?"
template = f'''<[INST] {prompt} [/INST]'''
print(llm(template))

 La rose est un terme qui peut désigner plusieurs choses.

Dans le contexte de l'anglais, "rose" peut faire référence à :

1. Une fleur appartenant à la famille des roses (Rosaceae), qui se caractérise par ses feuilles et ses fleurs colorées.
2. Un nom de famille, notamment porté par les personnes suivantes :
* Pierre-Louis Rose : un homme politique français du XIXe siècle ;
* Jean Rose : un footballeur français né en 1950;
* Nathalie Rose : une présentatrice française née en 1973.
1. Un nom de ville, notamment porté par les personnes suivantes :
* Rose : une ville de la province du Quebec au Canada.


In [ ]:
prompt = "Write code in Java to multiply three number"
template = f'''<[INST] {prompt} [/INST]'''
print(llm(template))

 ```java
public class MultiplyThreeNumbers {
    public static void main(String[] args) {
        int num1 = 5; //First Number
        int num2 = 6; //Second Number
        int num3 = 7; //Third Number
        
        int result = num1 * num2 * num3; //Multiplying the three numbers
        
        System.out.println("Product of " + num1 + ", " + num2 + " and " + num3 + " is: " + result);
    }
}
```


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.1 M

In [ ]:
import gradio as gr
from ctransformers import AutoModelForCausalLM, AutoConfig, Config

# Configure the model settings
conf = AutoConfig(Config(temperature=0.7, repetition_penalty=1.1, batch_size=52,
                         max_new_tokens=1024, context_length=2048))

# Load the model with the specified configuration
llm = AutoModelForCausalLM.from_pretrained("/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                                           model_type="mistral", config=conf)

# Define the function to generate a response using the model
def generate_response(prompt):
    template = f"<[INST] {prompt} [/INST]>"
    response = llm(template)  # Generate the response
    return response

# Set up the Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question"),  # Use gr.Textbox directly
    outputs=gr.Textbox()  # Use gr.Textbox for output as well
)

# Launch the Gradio app
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6739fd30a7e8c764f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
